In [1]:
# Module Importations
import numpy as np
import pandas as pd
import sklearn
import time
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

# Print versioning information
print(sklearn.__version__)

0.22.2.post1


In [2]:
# Project Module Importations
from src.data import load_data
from src.data import split_data
from src.models import model_evaluation
from src.models import sklearn_helpers 

In [3]:
# Load dataset
original_dataset = load_data.load_motor_data()

In [4]:
# Split data into training / evaluation sets
training_set, evaluation_set = split_data.split_train_test(original_dataset, 0.8)

In [5]:
# Drop profile id data (both sets)
training_set = training_set.drop("profile_id", axis = 1)
print(training_set.describe())

evaluation_set = evaluation_set.drop("profile_id", axis = 1)

ambient        coolant            u_d            u_q  \
count  199614.000000  199614.000000  199614.000000  199614.000000   
mean       -0.005043       0.005308       0.003792      -0.005359   
std         0.996490       1.002488       0.998747       1.001464   
min        -5.239872      -1.270450      -1.654002      -1.852853   
25%        -0.601386      -1.037872      -0.831843      -0.924537   
50%         0.266671      -0.178200       0.267391      -0.098490   
75%         0.686838       0.673512       0.358587       0.848608   
max         2.954662       2.296845       2.273808       1.788773   

         motor_speed         torque            i_d            i_q  \
count  199614.000000  199614.000000  199614.000000  199614.000000   
mean       -0.004852      -0.001687       0.004184      -0.001570   
std         1.001954       0.998525       0.999021       0.998389   
min        -1.353747      -3.339106      -3.235659      -3.329749   
25%        -0.951892      -0.267419      -0.75

In [6]:
# Create targets dataset (Stator yoke, tooth and winding temperature)

# Training Set
stator_training_data = training_set.drop(["stator_yoke", "stator_tooth", "stator_winding"], axis = 1)

stator_label_data = training_set[["stator_yoke", "stator_tooth", "stator_winding"]].copy()

# Evaluation Set
stator_eval_data = evaluation_set.drop(["stator_yoke", "stator_tooth", "stator_winding"], axis = 1)

stator_eval_label = evaluation_set[["stator_yoke", "stator_tooth", "stator_winding"]].copy()

In [8]:
# Evaluation metric (mean of ranges)

def return_column_range(dataframe, column_name):
    column_range = dataframe[column_name].max() - dataframe[column_name].min()
    return column_range

def ranges_mean(ranges_list):
    ranges_mean_value = sum(ranges_list) / len(ranges_list)
    return ranges_mean_value

# Claculate mean of each data column
ranges_sum = []

range_yoke = return_column_range(stator_label_data, "stator_yoke")
ranges_sum.append(range_yoke)

range_tooth = return_column_range(stator_label_data, "stator_tooth")
ranges_sum.append(range_tooth)

range_winding = return_column_range(stator_label_data, "stator_winding")
ranges_sum.append(range_winding)
print(ranges_sum)

# Claculate mean of ranges
ranges_mean = ranges_mean(ranges_sum)
print(ranges_mean)

[4.2768993, 4.3885418000000005, 4.6529601]
4.4394670666666665


In [9]:
# Linear Regression Model (Target - Yoke/Tooth/Winding Temperature)
lin_reg = LinearRegression()
lin_reg.fit(stator_training_data, stator_label_data)

# Name model
model_type = "[MO]LinRegr"
model_name = sklearn_helpers.name_model(model_type)

# Save model
sklearn_helpers.save_model(lin_reg, model_name)

# Compute RMSE via cross valiation
scores_mse = cross_val_score(lin_reg, stator_training_data, stator_label_data, scoring = "neg_mean_squared_error", cv = 5)
scores_rmse = np.sqrt(-scores_mse)

# Compute MAE via cross validation
scores_mae = cross_val_score(lin_reg, stator_training_data, stator_label_data, scoring = "neg_mean_absolute_error", cv = 5)
scores_mae = -1 * scores_mae

# Print CV results
print(model_type, "rmse mean (cv):", scores_rmse.mean())
print(model_type, "rmse std (cv):", scores_rmse.std())
print(model_type, "Percent Err (cv):", ((scores_rmse.mean()/ranges_mean)*100))

print(model_type, "mae mean (cv):", scores_mae.mean())
print(model_type, "mae std (cv):", scores_mae.std())
print(model_type, "Percent Err (cv):", ((scores_mae.mean()/ranges_mean)*100))

# Evaluation
start_time = time.process_time()
stator_pred_eval = lin_reg.predict(stator_eval_data)
end_time = time.process_time()
print("Prediction Time:", (end_time-start_time))

model_evaluation.evaluate_model(model_type, stator_eval_label, stator_pred_eval)

Save Path: C:\Developer\electric_motor_thermal_modelling\Models\YC_[MO]LinRegr_2020_08_11-11_02_22.pkl
[MO]LinRegr rmse mean (cv): 0.38052400693843447
[MO]LinRegr rmse std (cv): 0.002518489673301722
[MO]LinRegr Percent Err (cv): 8.571389340751377
[MO]LinRegr mae mean (cv): 0.27681278148007504
[MO]LinRegr mae std (cv): 0.0013704624881080666
[MO]LinRegr Percent Err (cv): 6.235270525115471
Prediction Time: 0.078125
[MO]LinRegr rmse (Eval): 0.38016795502001005
[MO]LinRegr mae (Eval): 0.2765778162024338
[MO]LinRegr r2 (Eval): 0.8551919747563934


In [39]:
# Decision Tree Regressor Model (Target - Yoke/Tooth/Winding Temperature)
tree_reg = DecisionTreeRegressor()
tree_reg.fit(stator_training_data, stator_label_data)

# Name model
model_type = "[MO]TreeRegr"
model_name = sklearn_helpers.name_model(model_type)

# Save model
sklearn_helpers.save_model(tree_reg, model_name)

# Compute RMSE via cross valiation
scores_mse = cross_val_score(tree_reg, stator_training_data, stator_label_data, scoring = "neg_mean_squared_error", cv = 5)
scores_rmse = np.sqrt(-scores_mse)

# Compute MAE via cross validation
scores_mae = cross_val_score(tree_reg, stator_training_data, stator_label_data, scoring = "neg_mean_absolute_error", cv = 5)
scores_mae = -1 * scores_mae

# Print CV results
model_name = "(Multi-Output) Decision Tree Regressor"
print(model_name, "rmse mean (cv):", scores_rmse.mean())
print(model_name, "rmse std (cv):", scores_rmse.std())
print(model_name, "Percent Err (cv):", ((scores_rmse.mean()/ranges_mean)*100))

print(model_name, "mae mean (cv):", scores_mae.mean())
print(model_name, "mae std (cv):", scores_mae.std())
print(model_name, "Percent Err (cv):", ((scores_mae.mean()/ranges_mean)*100))

# Evaluation
start_time = time.process_time()
stator_pred_eval = tree_reg.predict(stator_eval_data)
end_time = time.process_time()
print("Prediction Time:", (end_time-start_time))

model_evaluation.evaluate_model(model_name, stator_eval_label, stator_pred_eval)

(Multi-Output) Decision Tree Regressor rmse mean (cv): 0.12072972832880532
(Multi-Output) Decision Tree Regressor rmse std (cv): 0.003910139730143068
(Multi-Output) Decision Tree Regressor Percent Err (cv): 2.7194644428222814
(Multi-Output) Decision Tree Regressor mae mean (cv): 0.033279959589592456
(Multi-Output) Decision Tree Regressor mae std (cv): 0.0006864421996982308
(Multi-Output) Decision Tree Regressor Percent Err (cv): 0.7496386185511319
(Multi-Output) Decision Tree Regressor rmse (Eval): 0.11091435219019949
(Multi-Output) Decision Tree Regressor mae (Eval): 0.028154961854548462
(Multi-Output) Decision Tree Regressor r2 (Eval): 0.9876740610641462


In [41]:
# Random Forest Regressor Model (Target - Yoke/Tooth/Winding Temperature)
forest_reg = RandomForestRegressor()
forest_reg.fit(stator_training_data, stator_label_data)

# Name model
model_type = "[MO]ForestRegr"
model_name = sklearn_helpers.name_model(model_type)

# Save model
sklearn_helpers.save_model(forest_reg, model_name)

# Compute RMSE via cross valiation
scores_mse = cross_val_score(forest_reg, stator_training_data, stator_label_data, scoring = "neg_mean_squared_error", cv = 5)
scores_rmse = np.sqrt(-scores_mse)

# Compute MAE via cross validation
scores_mae = cross_val_score(forest_reg, stator_training_data, stator_label_data, scoring = "neg_mean_absolute_error", cv = 5)
scores_mae = -1 * scores_mae

# Print CV results
model_name = "(Multi-Output) Random Forest Regressor"
print(model_name, "rmse mean (cv):", scores_rmse.mean())
print(model_name, "rmse std (cv):", scores_rmse.std())
print(model_name, "Percent Err (cv):", ((scores_rmse.mean()/ranges_mean)*100))

print(model_name, "mae mean (cv):", scores_mae.mean())
print(model_name, "mae std (cv):", scores_mae.std())
print(model_name, "Percent Err (cv):", ((scores_mae.mean()/ranges_mean)*100))

# Evaluation
start_time = time.process_time()
stator_pred_eval = forest_reg.predict(stator_eval_data)
end_time = time.process_time()
print("Prediction Time:", (end_time-start_time))

model_evaluation.evaluate_model(model_name, stator_eval_label, stator_pred_eval)

(Multi-Output) Random Forest Regressor rmse mean (cv): 0.08195011489006596
(Multi-Output) Random Forest Regressor rmse std (cv): 0.0018287740887306444
(Multi-Output) Random Forest Regressor Percent Err (cv): 1.8459448771538576
(Multi-Output) Random Forest Regressor mae mean (cv): 0.03131729587239264
(Multi-Output) Random Forest Regressor mae std (cv): 0.00033060547953988505
(Multi-Output) Random Forest Regressor Percent Err (cv): 0.7054291743154419
(Multi-Output) Random Forest Regressor rmse (Eval): 0.07456924123853544
(Multi-Output) Random Forest Regressor mae (Eval): 0.027295399097267747
(Multi-Output) Random Forest Regressor r2 (Eval): 0.9944285533635128
